<a href="https://colab.research.google.com/github/evakaldy/Bootcamp_Final_Project/blob/eva_database_exploration/explore_database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# Krok 1: Připojte Google Disk
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
# Krok 2: Cesta k databázi na Google Disku
# Nahraďte 'path_to_your_database.db' skutečnou cestou k databázovému souboru na Google Disku
db_path = '/content/drive/MyDrive/Colab Notebooks/INTENSIVE_part_offline/final_project/data.db'


In [16]:
import sqlite3
import pandas as pd


# Connect to a SQLite database (or create one)
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

In [17]:
# Výpis všech tabulek
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Výpis názvů tabulek
for table in tables:
    print(table[0])

Accounts
sqlite_sequence
Campaigns
Clients
Outcomes


In [19]:
accounts_df = pd.read_sql_query("select * from Accounts", conn)
accounts_df

,id,client_id,in_default,balance,housing,loan,contact
0,1,4815378,no,2143.0,yes,no,unknown
1,2,5217541,no,29.0,yes,no,unknown
2,3,6654642,no,2.0,yes,yes,unknown
3,4,1421898,no,1506.0,yes,no,unknown
4,5,2387933,no,1.0,no,no,unknown
...,...,...,...,...,...,...,...
45206,45207,6145087,no,825.0,no,no,cellular
45207,45208,8051149,no,1729.0,no,no,cellular
45208,45209,9198356,no,5715.0,no,no,cellular
45209,45210,3187020,no,668.0,no,no,telephone
